# MPI single wrap

Lest start from imports

In [ ]:
import os
from jobqueue_features import (
    MPIEXEC,
    CustomSLURMCluster,
)

from jobqueue_features import mpi_wrap, on_cluster, mpi_task

Let's defince task script path.

In [ ]:
script_path = os.path.abspath(
    os.path.join(os.getcwd(), "resources", "helloworld.py")
)

Let's define cluster configuration.

In [ ]:
common_kwargs = {
    "walltime": "00:04:00",
    "cores_per_node": 2,
    "minimum_cores": 2,
    "hyperthreading_factor": 1,
    "ntasks_per_node": 2,
    "memory": "256 MB",
    "mpi_mode": True,
    "env_extra": [
        "export OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1",
        "export OMPI_ALLOW_RUN_AS_ROOT=1",
    ],
    "mpi_launcher": MPIEXEC,
    "local_directory": "/tmp",
    "queue": "batch",
}

Let's start cluster.

In [ ]:
fork_cluster = CustomSLURMCluster(
    name="fork_cluster",
    fork_mpi=True,
    nodes=2,
    **common_kwargs
)

Let's define task.

In [ ]:
@on_cluster(cluster=fork_cluster)
@mpi_task(cluster_id=fork_cluster.name)
def mpi_wrap_task(**kwargs):
    return mpi_wrap(**kwargs)

First check if we can construct the command:

In [ ]:
result = mpi_wrap_task(
    executable="python",
    exec_args=script_path,
    return_wrapped_command=True
)
result = result.result()
print(result)

Then check the execution of it:

In [ ]:
result = mpi_wrap_task(
    executable="python",
    exec_args=script_path
)
result = result.result()
print(result["out"])

It's alive!